<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.01/bds_yfinance_001_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Yahoo%20Finance.png?raw=true" width="150" height="">
</p>


 # **<font color="DeepPink">Datos remotos con yfinance</font>**

 ## **<font color="DeepPink">Instalamos y cargarmos las bibliotecas necesarias</font>**

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf #para obtener las series de precios históricos del fondo
import pandas_datareader.data as web #para obtener los factores del modelo de Fama & French
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go

 ## **<font color="DeepPink">Obtención remota de datos financieros</font>**

In [ ]:
end = dt.date(2022, 12, 31)
start = dt.date(end.year - 5, end.month, end.day)
fund = ['FDGRX']

👀 [Fidelity® Growth Company (FDGRX)](https://fundresearch.fidelity.com/mutual-funds/summary/316200104?type=sq-NavBar) es un fondo de inversiones que contiene más 500 compañías.

In [ ]:
Fund = yf.download(fund, start, end, interval = '1mo', rounding='true')
Fund.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-01,18.15,19.81,18.15,19.66,13.75,0
2018-02-01,19.54,19.70,17.99,19.28,13.49,0
2018-03-01,19.00,20.10,18.50,18.80,13.15,0
2018-04-01,18.19,19.43,18.19,18.81,13.16,0
2018-05-01,18.93,19.90,18.89,19.85,13.89,0


<p align="justify">
👀 Ahora descargamos los datos correspondientes a la cotización de cierre ajustado. Es decir, vamos a tener una serie de datos.</p>

In [ ]:
priceFund = yf.download(fund, start, end, interval = '1mo', rounding='true')['Adj Close']
priceFund.head()

[*********************100%%**********************]  1 of 1 completed


Date
2018-01-01    13.75
2018-02-01    13.49
2018-03-01    13.15
2018-04-01    13.16
2018-05-01    13.89
Name: Adj Close, dtype: float64

In [ ]:
type(priceFund)

pandas.core.series.Series

In [ ]:
priceFund.shape

(60,)

<p align="justify">
👀 Y ahora calculamos el cambio porcentual.</p>

In [ ]:
retFund = priceFund.pct_change()[1:]
retFund.shape

(59,)

 ## **<font color="DeepPink">Análisis gráfico de precios</font>**

In [ ]:
Fund.reset_index(inplace=True)

In [ ]:
fig = px.line(Fund,
              x='Date',
              y='Adj Close',
              template="gridon",
              markers=True,
              title="Fund's price evolution")

fig.show()

In [ ]:
fig = px.line(Fund,
              x = "Date",
              y = "Adj Close",
              template="gridon",
              markers=True)

fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1A", step="year", stepmode="backward"),
            dict(label = "Máx.", step="all")])))

fig.show()

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=Fund["Date"],
                                     open=Fund["Open"],
                                     high=Fund["High"],
                                     low=Fund["Low"],
                                     close=Fund["Close"])])

fig.update_layout(xaxis_rangeslider_visible=True, template="gridon")

fig.show()

 ## **<font color="DeepPink">Obtención remota de las variables del modelo de Fama & French</font>**

<p align="justify">
El modelo de tres factores Fama-French es una extensión del modelo de valoración de activos de capital (CAPM por sus siglas en inglés). El modelo Fama-French tiene como objetivo explicar los rendimientos de las acciones a través de tres factores:</p>

- (1) prima por riesgo de mercado,
- (2) el rendimiento superior de las empresas de pequeña capitalización en relación con las empresas de gran capitalización bursatil, y
- (3) el rendimiento superior de las empresas de valor (valor libro - valor de mercado alto) en relación a las empresas de crecimiento (valor libro - valor de mercado bajo).

<p align="justify">
La lógica detrás del modelo es que las empresas de valor y de pequeña capitalización bursatil tienden a superar regularmente en cuanto a rendimiento esperado a las empresas de crecimiento y alta capitalización bursatil.<p>

La fórmula del modelo de tres factores es:

$$r_i = r_f + \beta_1(r_m - r_f) + \beta_2(SMB)+\beta_3(HML)+ϵ_i$$

donde:
* $r_i$  = tasa de rendimiento esperada
* $rf$ = tasa libre de riesgo
* $\beta$  = coeficiente del factor (sensibilidad)
* $(r_m – r_f)$ = prima de riesgo de mercado
* $SMB$ (Small Minus Big) = rendimiento excedente histórico de las empresas de pequeña capitalización bursatil sobre las empresas de gran capitalización. Efecto tamaño.
* $HML$ (High Minus Low) = exceso de rendimiento histórico de las acciones de valor sobre las acciones de crecimiento
* $\epsilon_i$  = error aleatorio

Más información sobre el modelo de Fama y French y la base de datos:
* [CFI: Fama-French Three-Factor Model](https://corporatefinanceinstitute.com/resources/valuation/fama-french-three-factor-model/)
* [Base de datos](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html)
* [Pandas DataReader](https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#fama-french)




In [ ]:
web.DataReader('F-F_Research_Data_Factors', 'famafrench', start, end) #genera un diccionario

{0:          Mkt-RF   SMB   HML    RF
 Date                             
 2017-12    1.06 -1.32  0.06  0.09
 2018-01    5.57 -3.13 -1.29  0.12
 2018-02   -3.65  0.25 -1.04  0.11
 2018-03   -2.35  4.06 -0.21  0.11
 2018-04    0.28  1.13  0.54  0.14
 ...         ...   ...   ...   ...
 2022-08   -3.77  1.40  0.29  0.19
 2022-09   -9.35 -0.81  0.05  0.19
 2022-10    7.83  0.06  8.01  0.23
 2022-11    4.60 -3.52  1.38  0.29
 2022-12   -6.41 -0.69  1.37  0.33
 
 [61 rows x 4 columns],
 1:       Mkt-RF    SMB    HML    RF
 Date                            
 2017   21.51  -5.09 -13.57  0.80
 2018   -6.95  -3.16  -9.67  1.83
 2019   28.28  -6.14 -10.46  2.15
 2020   23.66  13.18 -46.67  0.45
 2021   23.56  -3.89  25.49  0.04
 2022  -21.60  -6.95  25.81  1.43,
 'DESCR': 'F-F Research Data Factors\n-------------------------\n\nThis file was created by CMPT_ME_BEME_RETS using the 202311 CRSP database. The 1-month TBill return is from Ibbotson and Associates, Inc. Copyright 2023 Kenneth R. French\n\

In [ ]:
factors = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start, end)[0]
factors.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
2017-12,1.06,-1.32,0.06,0.09
2018-01,5.57,-3.13,-1.29,0.12
2018-02,-3.65,0.25,-1.04,0.11
2018-03,-2.35,4.06,-0.21,0.11
2018-04,0.28,1.13,0.54,0.14


 ## **<font color="DeepPink">Combinamos los 2 conjuntos de datos</font>**

In [ ]:
factors = factors[2:]
factors.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
2018-02,-3.65,0.25,-1.04,0.11
2018-03,-2.35,4.06,-0.21,0.11
2018-04,0.28,1.13,0.54,0.14
2018-05,2.65,5.25,-3.20,0.14
2018-06,0.48,1.14,-2.34,0.14


In [ ]:
retFund.index = factors.index
retFund.head()

Date
2018-02   -0.018909
2018-03   -0.025204
2018-04    0.000760
2018-05    0.055471
2018-06    0.001440
Freq: M, Name: Adj Close, dtype: float64

In [ ]:
len(retFund)==len(factors)

True

In [ ]:
fullTable = pd.merge(retFund, factors, on="Date")

In [ ]:
fullTable.head()

,Adj Close,Mkt-RF,SMB,HML,RF
Date,,,,,
2018-02,-0.018909,-3.65,0.25,-1.04,0.11
2018-03,-0.025204,-2.35,4.06,-0.21,0.11
2018-04,0.000760,0.28,1.13,0.54,0.14
2018-05,0.055471,2.65,5.25,-3.20,0.14
2018-06,0.001440,0.48,1.14,-2.34,0.14


In [ ]:
fullTable[['Mkt-RF','SMB','HML','RF']]=fullTable[['Mkt-RF','SMB','HML','RF']]/100

In [ ]:
fullTable.head()

,Adj Close,Mkt-RF,SMB,HML,RF
Date,,,,,
2018-02,-0.018909,-0.0365,0.0025,-0.0104,0.0011
2018-03,-0.025204,-0.0235,0.0406,-0.0021,0.0011
2018-04,0.000760,0.0028,0.0113,0.0054,0.0014
2018-05,0.055471,0.0265,0.0525,-0.0320,0.0014
2018-06,0.001440,0.0048,0.0114,-0.0234,0.0014


In [ ]:
fullTable.rename(columns={"Adj Close":"Ri"}, inplace=True)

In [ ]:
fullTable['Ri-RF'] = fullTable['Ri']-fullTable['RF']

In [ ]:
fullTable.head()

,Ri,Mkt-RF,SMB,HML,RF,Ri-RF
Date,,,,,,
2018-02,-0.018909,-0.0365,0.0025,-0.0104,0.0011,-0.020009
2018-03,-0.025204,-0.0235,0.0406,-0.0021,0.0011,-0.026304
2018-04,0.000760,0.0028,0.0113,0.0054,0.0014,-0.000640
2018-05,0.055471,0.0265,0.0525,-0.0320,0.0014,0.054071
2018-06,0.001440,0.0048,0.0114,-0.0234,0.0014,0.000040


In [ ]:
fullTable.drop(columns=["Ri", "RF"], inplace=True)

In [ ]:
fullTable.head()

,Mkt-RF,SMB,HML,Ri-RF
Date,,,,
2018-02,-0.0365,0.0025,-0.0104,-0.020009
2018-03,-0.0235,0.0406,-0.0021,-0.026304
2018-04,0.0028,0.0113,0.0054,-0.000640
2018-05,0.0265,0.0525,-0.0320,0.054071
2018-06,0.0048,0.0114,-0.0234,0.000040


**... y queda listo nuestro DataFrame!**